# Main Challenge: Variational Quantum Unsampling

Here we provide you some introductory material to get you started on the main challenge, 
the results of which you will be presenting tomorrow.

In this challenge, you will be investigating a recently defined algorithm,
called 'Variational Quanutm Unsampling' 

(https://arxiv.org/abs/1904.10463)

This is a variational algorithm, whose goal is to determine which process created a particular 
quantum state which is fed into it.

Essentially, it tries to determine some unitary, $U$, which created the state:

$\left|\psi_{\text{out}}\right> = U\left|\psi_{\text{in}}\right>$

when applied to some (known) reference state, $\left|\psi_{\text{in}}\right>$

The algorithm does this by applying some parameterised unitary operation, $V(\theta)$, on the state $\left|\psi_{\text{out}}\right>$ in order to return it to the reference state, $\left|\psi_{\text{in}}\right>$. In other words, it attempts to find the inverse of the target unitary, $U$, such that:

$\left|\psi_{\text{in}}\right> =  V(\theta)U\left|\psi_{\text{in}}\right>$

This is achieved by minimising some cost function, relative to the parameters, $\theta$, which describes the discrepancy between the target, and the current state of the system at any point during the optimisation procedure.

First import the usual things:

In [1]:
from pyquil import Program
from pyquil.api import get_qc, WavefunctionSimulator, local_qvm
from pyquil.gates import *
import numpy as np
import os, inspect, sys

import sys
sys.path.insert(0, 'tests/')

make_wf = WavefunctionSimulator()

# Practicalities:

Actually optimising the parameters, $\theta$, of an $n$-qubit state properly would be extremely difficult, if all the measurements were performed at the end. 

Instead a solution is to break the unitary, $V(\theta)$ into a number of smaller unitaries:

$V_n(\vec{\theta}_n), V_{n-1}(\vec{\theta}_{n-1}), \dots V_1(\vec{\theta}_1)$, where $V_i$ acts on $i$ qubits only.

After each unitary is performed, a *single* qubit is measured, and the measurement results are used to optimise the parameters of said unitary.


# Loss Function

As is standard in Machine Learning, we must define some loss function to determine how well we are doing during training.

In this case, the loss function will tell us how close the output was to the input, measured by the inner product of the two states.

Firslty, imagine we did not break up the unitary, $V(\theta)$, which evolved the output states, $\left|\psi_{out}\right>$, to some final state:

$\left|\phi\right> = V(\theta)\left|\psi_{\text{out}}\right>$

In this case, we want to see how well we are doing, by measuring the *squared overlap* between the states $\left|\psi_{\text{out}}\right>$ and the target states, $\left|\psi_{\text{in}}\right>$:

$L = 1 - \left|\left<\phi|\psi_{\text{out}}\right>\right|^2$.

This is the same as taking the *inner product* between the vectors $\left|\phi\right>, \left|\psi_{\text{out}}\right>$. If these vectors are identical, i.e. $\left|\phi\right> = \left|\psi_{\text{out}}\right>$, then their inner product will be $1$.
In this case, the above cost function, $L$, will be $0$ and we have achieved an optimal minimum.

Unfortunately, as mentioned above, this overlap may be exponentially small, i.e. the probability of ending in the
state $\left|\psi_{\text{in}}\right>$ may be very small, especially as the number of qubits grows.

A solution is to break up the unitary to the smaller ones above, and sequentially minimise a *sequence* of loss functions,one for each qubit that is measured.

For example, if we assume that our input state was a product state: $\left|\vec{\alpha}\right> = \left|\alpha_1\right>\otimes\dots \lef\alpha_n\right>$, then we can optimise per single qubit state, $\left|\alpha_i\right>$

$L_i = 1 - \left|\left<\alpha_i\right|\mathsf{Tr}_{-i}\left(\rho\right)\left|\alpha_i\right>\right|^2$

**Note: $\mathsf{Tr}_{-i}(\rho)$ denotes the partial trace of a quantum density matrix, $\rho$, including all subsystems except $i$**

*Example: if we have a pure state of three qubits, $\left|\psi\right> = \left|a\right> \otimes\left|b\right> \otimes\left|c\right>$, which has a corresponding density matrix*:

*$\rho = \left|a\right>\left<a\right| \otimes\left|b\right>\left<b\right| \otimes\left|c\right>\left<c\right|$* (for pure states, the density matrix is given by the outer product of $\left|\psi\right>$ with itself, $\rho = \left|\psi\right>\left<\psi\right|$

*Then the partial trace over all but system $b$ is:*

 $\mathsf{Tr}_{-b}(\rho) = \left(\left|a\right>\left<a\right| \otimes\left|b\right>\left<b\right| \otimes \left|c\right>\left<c\right|\right) =  \left|b\right>\left<b\right| \mathsf{Tr}\left(\left|a\right>\left<a\right| \otimes \left|c\right>\left<c\right|\right) = \left|b\right>\left<b\right| \mathsf{Tr}\left(\left<a|a\right> \otimes \left<c|c\right>\right) = \left|b\right>\left<b\right|$


# How does the algorithm work?

We will use a specific case of the algorithm and consider $\left|\psi_{\text{in}}\right> = |00\dots 0\rangle$ to be a product state.
## Input:
A state prepared by some unknown unitary, $U|00\dots 0\rangle$

For each round, $i: n \rightarrow 1$, apply $i^{th}$ parameterised unitary, $V(\vec{\theta}_i)$, and measure $i^{th}$ qubit fidelity from $\|0\rangle$ state. Compute Loss function, L_i, and update parameters, $\vec{\theta}_i$ using some minimiser.

## Output:
Optimal angles, $\vec{\theta}_n, \dots, \vec{\theta}_1$ to approximate target unitary, $U$.


We start with a simple example, the two qubit bell state, prepared by the following circuit:

$U_{Bell} = CNOT_{0,1}(H \otimes \mathbb{1})|00\rangle$
    

In [8]:
circuit = Program()

qc_name = "3q-qvm"
qc =get_qc(qc_name)
qubits = qc.qubits()
print(qc.qubits())

# with local_qvm():
#     wavefunc = make_wf.wavefunction(circuit)
    
# print(wavefunc)


[0, 1, 2]


We have prepared the target unitary, now we must apply a parameterised two qubit unitary, to find the CNOT.

We will use a Control-Rx gate for the sake of argument: We want to ensure firstly that the algorithm **can** find the appropriate angles: For a specific setting of the parameter in the Rx gate, we recover the CNOT.



We need to define the Control-X gate, since it is not built into Pyquil:
(http://docs.rigetti.com/en/stable/basics.html#defining-parametric-gates)

In [9]:
from pyquil.parameters import Parameter, quil_sin, quil_cos
from pyquil.quilbase import DefGate

# Define the new gate from a matrix
theta = Parameter('theta')
crx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -1j * quil_sin(theta / 2)],
    [0, 0, -1j * quil_sin(theta / 2), quil_cos(theta / 2)]
])

gate_definition = DefGate('CRX', crx, [theta])
CRX = gate_definition.get_constructor()



In [10]:
# Create our program and use the new parametric gate
# print(circuit)
with local_qvm():
    wavefunc = make_wf.wavefunction(circuit)


# print(wavefunc)

Now we measure qubit $1$ and try and maximise the probability that it is the $|0\rangle$ state.

In [25]:
num_trials = 1000
params = np.random.rand(1) # We will start with two random parameters
print(params)
import scipy.optimize as opt
print(circuit)

circuit.wrap_in_numshots_loop(num_trials)


def cost_func_n(params, circuit, qubits):
    circuit = Program()
    circuit += gate_definition # We need to define the new gate in each program we use it in.

    ro = circuit.declare('ro', 'BIT', 2)

    circuit += H(qubits[0])
    circuit += CNOT(qubits[0], qubits[1])

    circuit += CRX(params[0])(qubits[0], qubits[1])
    circuit += MEASURE(qubits[1], ro[0])
    print(circuit)

    executable = qc.compile(circuit)
    results = qc.run(executable)
    prob_zero = list(results).count([0])/num_trials
    cost_func = 1 - prob_zero

    return cost_func

result = opt.minimize(cost_func_n, params, args=(circuit, qubits), method='Powell')

print(result)

[0.32829278]
DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)


DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

DECLARE ro BIT[2]
H 0
CNOT 0 1
CRX(0.3282927785808821) 0 1
MEASURE 1 ro[0]

DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

DECLARE ro BIT[2]
H 0
CNOT 0 1
CRX(0.3282927785808821) 0 1
MEASURE 1 ro[0]

DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

DECLARE ro BIT[2]
H 0
CNOT 0 1
CRX(1.3282927785808822) 0 1
MEASURE 1 ro[0]

DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

DECLARE ro BIT[2]
H 0
CNOT 0 1
CRX(-1.289741221419118) 0 1
MEASURE 1 ro[0]

DE

DEFGATE CRX(%theta):
    1, 0, 0, 0
    0, 1, 0, 0
    0, 0, COS(%theta/2), -i*SIN(%theta/2)
    0, 0, -i*SIN(%theta/2), COS(%theta/2)

DECLARE ro BIT[2]
H 0
CNOT 0 1
CRX(-5.926792641626021) 0 1
MEASURE 1 ro[0]

   direc: array([[-1.53433171]])
     fun: array(0.999)
 message: 'Optimization terminated successfully.'
    nfev: 39
     nit: 2
  status: 0
 success: True
       x: array(-5.89498673)
